In [ ]:
import pandas as pd
pd.set_option("display.max_columns",False)

In [ ]:
pip install openpyxl

In [ ]:
df=pd.read_excel("/kaggle/input/finala/zomato_final.xlsx",sheet_name="Sheet1")

In [ ]:
import ast
import ast
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
from collections import Counter


In [ ]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
# Load model and tokenizer
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def sentiment(x):
    reviews_list = ast.literal_eval(x)[:20]
    sentiments=[]
    reviews = [review[1] for review in reviews_list]
#     print(type(reviews))
# Preprocess text (username and link placeholders)




    # Analyze sentiment for each review
    for i, review in enumerate(reviews):
        text = preprocess(review)
#         print(review)
        encoded_input = tokenizer(text, return_tensors='pt',max_length=512)
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)

        ranking = np.argsort(scores)[::-1]
        sentiment = config.id2label[ranking[0]]
        score = scores[ranking[0]]
        sentiments.append(sentiment)
    return(sentiments)

    
    


In [ ]:
def freq(l):
    return Counter(l)

In [ ]:
df1=df[5000:15000]

In [ ]:
def process_group(group):
    # Apply sentiment function to the first 20 rows of the group
    sentiments = group.head(20)['reviews_list'].apply(sentiment)
    
    # Flatten the list of lists to a single list
    all_sentiments = [item for sublist in sentiments for item in sublist]
    
    # Get the frequency of each sentiment
    sentiment_counts = freq(all_sentiments)
    
    # Extract the counts of 'positive', 'negative', and 'neutral'
    positive = sentiment_counts.get('positive', 0)
    negative = sentiment_counts.get('negative', 0)
    neutral = sentiment_counts.get('neutral', 0)
    
    return pd.Series([positive, negative, neutral], index=['positive', 'negative', 'neutral'])

# Apply the process_group function to each group
result = df1.groupby('name').apply(process_group)

In [ ]:
result.to_csv("sentiments.csv")